# Method 3: Local DP via GRR (per-individual single-SNP reporting)

Implements the LDP GRR mechanism where each CASE_POP case individual reports a single SNP genotype (0/1/2) with generalized randomized response. Debiases per-SNP genotype distributions to estimate case MAF.


In [1]:

import json
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt


# Robust PROJECT_ROOT detection


def find_project_root(start=None) -> Path:
    p = Path(start or Path.cwd()).resolve()
    for parent in [p] + list(p.parents):
        if (parent / ".git").exists() or (parent / "requirements.txt").exists():
            return parent
    return p

PROJECT_ROOT = find_project_root()
PROC_DIR = PROJECT_ROOT / "data" / "processed" / "hapmap"
REGIONS_DIR = PROC_DIR / "regions"
COHORTS_JSON = PROC_DIR / "cohorts" / "hapmap_case_pop.json"

DERIVED_DIR = PROJECT_ROOT / "data" / "derived" / "method3"
FIG_DIR = PROJECT_ROOT / "results" / "figures" / "method3"
TABLE_DIR = PROJECT_ROOT / "results" / "tables" / "method3"

for d in [DERIVED_DIR, FIG_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

if not COHORTS_JSON.exists():
    raise FileNotFoundError(f"Missing case_pop config: {COHORTS_JSON}")

case_info = json.loads(COHORTS_JSON.read_text())
CASE_POP = case_info["case_pop"]

EPS_LIST = [0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
SEED_LIST = list(range(30))
PVALUE_CUTOFF = 1e-3
ALPHA_TEST_EXCEED = 0.05
P_CLIP = 1e-6
REPORT_K = 1

REGION_FILES = {}
for region_tag, info in case_info["regions"].items():
    REGION_FILES[region_tag] = {
        "ceu": PROJECT_ROOT / info["ceu_region"],
        "case": PROJECT_ROOT / info["case_region"],
        "ceu_control_npz": PROJECT_ROOT / info["ceu_control_npz"],
    }

print("PROJECT_ROOT =", PROJECT_ROOT)
print("CASE_POP =", CASE_POP)
print("COHORTS_JSON =", COHORTS_JSON)
print("EPS_LIST =", EPS_LIST)
print("SEED_LIST =", SEED_LIST)
print("REPORT_K =", REPORT_K)
print("Chance baseline LR power ~", ALPHA_TEST_EXCEED)


PROJECT_ROOT = /Users/erkmenerken/Desktop/proje430
CASE_POP = MKK
COHORTS_JSON = /Users/erkmenerken/Desktop/proje430/data/processed/hapmap/cohorts/hapmap_case_pop.json
EPS_LIST = [0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
SEED_LIST = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
REPORT_K = 1
Chance baseline LR power ~ 0.05


In [2]:


# Load regions + cohorts

def load_region_npz(path: Path) -> Dict[str, np.ndarray]:
    if not path.exists():
        raise FileNotFoundError(f"Missing region file: {path}")
    z = np.load(path, allow_pickle=True)
    return {k: z[k] for k in z.files}


def load_indices_npz(path: Path) -> List[int]:
    if not path.exists():
        raise FileNotFoundError(f"Missing cohort file: {path}")
    z = np.load(path, allow_pickle=True)
    return list(z["indices"].astype(int))


def _pick_genotype_matrix(region: Dict[str, np.ndarray]) -> np.ndarray:
    for key in ["G", "G_sub", "genotypes", "X"]:
        if key in region:
            G = region[key]
            if isinstance(G, np.ndarray) and G.ndim == 2:
                return G
    for k, v in region.items():
        if isinstance(v, np.ndarray) and v.ndim == 2:
            return v
    raise ValueError("No 2D genotype matrix found in region npz.")


def _ensure_snps_by_individuals(G: np.ndarray) -> np.ndarray:
    if G.shape[0] < G.shape[1]:
        return G.T
    return G

regions = {}
ceu_control_idx = {}
case_pool_idx = {}

for region_tag, paths in REGION_FILES.items():
    regions[region_tag] = {
        "ceu": load_region_npz(paths["ceu"]),
        "case": load_region_npz(paths["case"]),
    }
    ceu_control_idx[region_tag] = load_indices_npz(paths["ceu_control_npz"])
    n_case = len(regions[region_tag]["case"]["sample_ids"])
    case_pool_idx[region_tag] = list(range(n_case))

    print(f"Loaded {region_tag} CEU: {paths['ceu'].relative_to(PROJECT_ROOT)}")
    print(f"Loaded {region_tag} CASE: {paths['case'].relative_to(PROJECT_ROOT)}")
    print(f"CEU control idx count: {len(ceu_control_idx[region_tag])} | CASE pool: {len(case_pool_idx[region_tag])}")


Loaded chr2_5Mb CEU: data/processed/hapmap/regions/CEU_chr2_5Mb.common_with_MKK.npz
Loaded chr2_5Mb CASE: data/processed/hapmap/regions/MKK_chr2_5Mb.common_with_CEU.npz
CEU control idx count: 165 | CASE pool: 171
Loaded chr10_1Mb CEU: data/processed/hapmap/regions/CEU_chr10_1Mb.common_with_MKK.npz
Loaded chr10_1Mb CASE: data/processed/hapmap/regions/MKK_chr10_1Mb.common_with_CEU.npz
CEU control idx count: 165 | CASE pool: 171


In [3]:

def compute_counts_and_maf(G_snps_x_ind: np.ndarray, cols: List[int]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    sub = G_snps_x_ind[:, cols]
    called = sub != -1
    called_n = called.sum(axis=1).astype(np.int32)
    sub_nonmiss = np.where(called, sub, 0)
    mac = sub_nonmiss.sum(axis=1).astype(np.float64)
    denom = 2.0 * np.maximum(called_n, 1)
    maf = mac / denom
    return called_n, mac, maf

def noisy_counts_from_maf(maf: np.ndarray, called_n: np.ndarray) -> np.ndarray:
    raw = np.rint(maf * (2.0 * called_n))
    return np.clip(raw, 0.0, 2.0 * called_n).astype(np.float64)

def chisq_pvalues_case_vs_control(mac_case: np.ndarray, called_case: np.ndarray,
                                 mac_ctrl: np.ndarray, called_ctrl: np.ndarray) -> np.ndarray:
    n = mac_case.shape[0]
    pvals = np.ones(n, dtype=np.float64)
    for j in range(n):
        minor_case = mac_case[j]
        major_case = 2.0 * called_case[j] - minor_case
        minor_ctrl = mac_ctrl[j]
        major_ctrl = 2.0 * called_ctrl[j] - minor_ctrl
        if called_case[j] <= 0 or called_ctrl[j] <= 0:
            pvals[j] = 1.0
            continue
        if major_case < 0 or major_ctrl < 0:
            pvals[j] = 1.0
            continue
        table = np.array([[minor_case, major_case], [minor_ctrl, major_ctrl]], dtype=np.float64)
        if np.any(table.sum(axis=1) <= 0) or np.any(table.sum(axis=0) <= 0):
            pvals[j] = 1.0
            continue
        try:
            _, p, _, _ = chi2_contingency(table, correction=False)
            pvals[j] = float(p)
        except Exception:
            pvals[j] = 1.0
    return pvals

def overlap_metrics(sig_true: np.ndarray, sig_noisy: np.ndarray) -> Dict[str, float]:
    a = sig_true.astype(bool)
    b = sig_noisy.astype(bool)
    tp = int(np.sum(a & b))
    fp = int(np.sum(~a & b))
    fn = int(np.sum(a & ~b))
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    jaccard = tp / (tp + fp + fn) if (tp + fp + fn) else 0.0
    return {
        "true_sig": int(np.sum(a)),
        "noisy_sig": int(np.sum(b)),
        "tp": tp,
        "precision": float(precision),
        "recall": float(recall),
        "jaccard": float(jaccard),
    }

def genotype_log_prob_hwe(g: np.ndarray, p: np.ndarray) -> np.ndarray:
    p = np.clip(p, P_CLIP, 1.0 - P_CLIP)
    q = 1.0 - p
    logP0 = 2.0 * np.log(q)
    logP1 = np.log(2.0) + np.log(p) + np.log(q)
    logP2 = 2.0 * np.log(p)
    out = np.zeros_like(p, dtype=np.float64)
    out = np.where(g == 0, logP0, out)
    out = np.where(g == 1, logP1, out)
    out = np.where(g == 2, logP2, out)
    return out

def lr_scores(G_snps_x_ind: np.ndarray, cols: List[int], p_case: np.ndarray, p_ctrl: np.ndarray) -> np.ndarray:
    G = G_snps_x_ind[:, cols]
    n_ind = G.shape[1]
    scores = np.zeros(n_ind, dtype=np.float64)
    for i in range(n_ind):
        g = G[:, i]
        mask = g >= 0
        if not np.any(mask):
            continue
        log_case = genotype_log_prob_hwe(g[mask], p_case[mask])
        log_ctrl = genotype_log_prob_hwe(g[mask], p_ctrl[mask])
        scores[i] = float(np.sum(log_case - log_ctrl))
    return scores

def split_train_test(indices: List[int], seed: int, frac_train: float = 0.7) -> Tuple[List[int], List[int]]:
    rng = np.random.default_rng(seed)
    shuffled = list(rng.permutation(indices))
    n = len(shuffled)
    n_train = int(round(n * frac_train))
    n_train = max(1, min(n - 1, n_train))
    return shuffled[:n_train], shuffled[n_train:]

def grr_probs(eps: float, k: int = 3) -> Tuple[float, float]:
    ee = np.exp(eps)
    p = ee / (ee + k - 1)
    q = (1.0 - p) / (k - 1)
    return p, q

def ldp_grr_estimate_maf(G_snps_x_ind: np.ndarray, train_idx: List[int], eps: float, rng: np.random.Generator,
                         report_k: int = 1, fallback_maf: np.ndarray = None) -> Tuple[np.ndarray, np.ndarray, float, float, int]:

    Gsub = G_snps_x_ind[:, train_idx]
    n_snps, n_ind = Gsub.shape
    p, q = grr_probs(eps, k=3)
    counts = np.zeros((n_snps, 3), dtype=np.int32)
    report_counts = np.zeros(n_snps, dtype=np.int32)

    k = min(report_k, n_snps)
    for i in range(n_ind):
        snps = rng.choice(n_snps, size=k, replace=False)
        for j in snps:
            g = int(Gsub[j, i])
            if g < 0:
                continue
            report_counts[j] += 1
            if rng.random() < p:
                r = g
            else:
                others = [0, 1, 2]
                others.remove(g)
                r = others[rng.integers(0, 2)]
            counts[j, r] += 1

    noisy_maf = np.zeros(n_snps, dtype=np.float64)
    zero_reports = 0
    for j in range(n_snps):
        m = report_counts[j]
        if m <= 0:
            zero_reports += 1
            noisy_maf[j] = fallback_maf[j] if fallback_maf is not None else 0.5
            continue
        f_hat = counts[j].astype(np.float64) / float(m)
        est = (f_hat - q) / (p - q)
        est = np.clip(est, 0.0, 1.0)
        s = est.sum()
        if s <= 0:
            est = np.array([1/3, 1/3, 1/3], dtype=np.float64)
        else:
            est = est / s
        noisy_maf[j] = (est[1] + 2.0 * est[2]) / 2.0

    noisy_maf = np.clip(noisy_maf, 0.0, 1.0)
    return noisy_maf, report_counts, p, q, zero_reports


In [4]:


# Main loop: Method 3 (LDP-GRR)


def eps_dir(eps: float) -> Path:
    s = str(eps).replace(".", "p")
    return DERIVED_DIR / f"eps_{s}"

records = []
hist_cache = {}

for region_tag, region_pair in regions.items():
    G_ceu = _ensure_snps_by_individuals(_pick_genotype_matrix(region_pair["ceu"]))
    G_case = _ensure_snps_by_individuals(_pick_genotype_matrix(region_pair["case"]))

    n_snps, n_ceu = G_ceu.shape
    n_snps_case, n_case = G_case.shape
    if n_snps != n_snps_case:
        raise RuntimeError(f"SNP count mismatch in {region_tag}: CEU={n_snps}, CASE={n_snps_case}")

    ceu_ctrl_idx = [i for i in ceu_control_idx[region_tag] if i < n_ceu]
    case_pool = [i for i in case_pool_idx[region_tag] if i < n_case]

    if len(ceu_ctrl_idx) == 0:
        raise RuntimeError(f"No CEU control indices for {region_tag}")
    if len(case_pool) == 0:
        raise RuntimeError(f"No CASE_POP indices for {region_tag}")

    print(f"[{region_tag}] n_snps={n_snps} | CEU ctrl={len(ceu_ctrl_idx)} | {CASE_POP} pool={len(case_pool)}")

    # Control MAF 
    called_ctrl, mac_ctrl, maf_ctrl = compute_counts_and_maf(G_ceu, ceu_ctrl_idx)

    for eps in EPS_LIST:
        outdir = eps_dir(eps)
        outdir.mkdir(parents=True, exist_ok=True)

        for seed in SEED_LIST:
            rng = np.random.default_rng(seed)
            case_train_idx, case_test_idx = split_train_test(case_pool, seed=seed, frac_train=0.7)

            # True case from CASE train
            called_case, mac_case, maf_case = compute_counts_and_maf(G_case, case_train_idx)

            # LDP-GRR release
            noisy_maf, report_counts, p_grr, q_grr, zero_reports = ldp_grr_estimate_maf(
                G_case, case_train_idx, eps, rng, report_k=REPORT_K, fallback_maf=maf_ctrl
            )
            noisy_mac_safe = noisy_counts_from_maf(noisy_maf, called_case)

            # Privacy: LR scores within CASE_POP
            scores_member = lr_scores(G_case, case_train_idx, p_case=noisy_maf, p_ctrl=maf_ctrl)
            scores_nonmember = lr_scores(G_case, case_test_idx, p_case=noisy_maf, p_ctrl=maf_ctrl)
            thresh = float(np.quantile(scores_nonmember, 1.0 - ALPHA_TEST_EXCEED))
            lr_power = float(np.mean(scores_member >= thresh))

            # Utility: GWAS overlap (true vs noisy)
            p_true = chisq_pvalues_case_vs_control(mac_case, called_case, mac_ctrl, called_ctrl)
            p_noisy = chisq_pvalues_case_vs_control(noisy_mac_safe, called_case, mac_ctrl, called_ctrl)
            sig_true = p_true <= PVALUE_CUTOFF
            sig_noisy = p_noisy <= PVALUE_CUTOFF
            ov = overlap_metrics(sig_true, sig_noisy)

            rmse = float(np.sqrt(np.mean((noisy_maf - maf_case) ** 2)))
            maf_mae = float(np.mean(np.abs(noisy_maf - maf_case)))
            corr = float(np.corrcoef(noisy_maf, maf_case)[0, 1]) if noisy_maf.size > 1 else 0.0

            
            seed_dir = outdir / f"seed_{seed}"
            seed_dir.mkdir(parents=True, exist_ok=True)
            out_path = seed_dir / f"{region_tag}.case_maf_ldp_{CASE_POP}.npz"
            np.savez_compressed(
                out_path,
                noisy_maf=noisy_maf,
                noisy_mac_safe=noisy_mac_safe,
                eps=float(eps),
                seed=int(seed),
                region_tag=region_tag,
                case_pop=CASE_POP,
                report_k=int(REPORT_K),
                p_grr=float(p_grr),
                q_grr=float(q_grr),
                zero_reports=int(zero_reports),
                report_counts=report_counts,
                case_train_idx=np.array(case_train_idx, dtype=np.int32),
                case_test_idx=np.array(case_test_idx, dtype=np.int32),
            )

            
            if abs(eps - 1.0) < 1e-9 and seed == 0:
                hist_cache[region_tag] = (scores_member, scores_nonmember, thresh)

            rec = {
                "seed": int(seed),
                "eps": float(eps),
                "region": region_tag,
                "case_pop": CASE_POP,
                "n_snps": int(n_snps),
                "n_train": int(len(case_train_idx)),
                "n_test": int(len(case_test_idx)),
                "n_ctrl": int(len(ceu_ctrl_idx)),
                "report_k": int(REPORT_K),
                "zero_reports": int(zero_reports),
                # privacy
                "lr_threshold_test_95pct": thresh,
                "lr_power": lr_power,
                "lr_member_mean": float(np.mean(scores_member)),
                "lr_nonmember_mean": float(np.mean(scores_nonmember)),
                # utility
                **ov,
                "maf_mae": maf_mae,
                "rmse_maf": rmse,
                "corr_maf": corr,
            }
            records.append(rec)

df_per_seed = pd.DataFrame(records)
df_per_seed = df_per_seed.sort_values(["region", "eps", "seed"]).reset_index(drop=True)

per_seed_csv = TABLE_DIR / f"method3_per_seed_{CASE_POP}.csv"
df_per_seed.to_csv(per_seed_csv, index=False)
print("Saved per-seed table ->", per_seed_csv.relative_to(PROJECT_ROOT))


[chr2_5Mb] n_snps=300 | CEU ctrl=165 | MKK pool=171
[chr10_1Mb] n_snps=588 | CEU ctrl=165 | MKK pool=171
Saved per-seed table -> results/tables/method3/method3_per_seed_MKK.csv


In [5]:

# Aggregate summary + plots


group_cols = ["region", "eps"]
agg_cols = ["lr_power", "jaccard", "maf_mae", "rmse_maf", "corr_maf", "true_sig", "noisy_sig"]

summary = df_per_seed.groupby(group_cols)[agg_cols].agg(["mean", "std"]).reset_index()
summary.columns = [c[0] if c[1] == "" else f"{c[0]}_{c[1]}" for c in summary.columns.to_flat_index()]

summary_csv = TABLE_DIR / f"method3_summary_{CASE_POP}.csv"
summary.to_csv(summary_csv, index=False)
print("Saved summary table ->", summary_csv.relative_to(PROJECT_ROOT))

for region_tag in summary["region"].unique():
    sub = summary[summary["region"] == region_tag].sort_values("eps")
    eps = sub["eps"].to_numpy()

    # Privacy curve
    plt.figure(figsize=(6, 4))
    plt.plot(eps, sub["lr_power_mean"], marker="o", label="LR power")
    plt.xscale("log")
    plt.xlabel("epsilon (log scale)")
    plt.ylabel("LR power (members above 95% nonmember threshold)")
    plt.title(f"Method 3 Privacy (LR power) — {region_tag}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{region_tag}.privacy_lr_power_{CASE_POP}.png")
    plt.close()

    # Utility curve
    plt.figure(figsize=(6, 4))
    plt.plot(eps, sub["jaccard_mean"], marker="o", label="Jaccard")
    plt.xscale("log")
    plt.xlabel("epsilon (log scale)")
    plt.ylabel("Jaccard(sig_true, sig_noisy)")
    plt.title(f"Method 3 Utility (GWAS overlap) — {region_tag}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{region_tag}.utility_jaccard_{CASE_POP}.png")
    plt.close()

    # DP distortion curve
    plt.figure(figsize=(6, 4))
    plt.plot(eps, sub["maf_mae_mean"], marker="o", label="MAF MAE")
    plt.xscale("log")
    plt.xlabel("epsilon (log scale)")
    plt.ylabel("MAF MAE")
    plt.title(f"Method 3 DP Distortion — {region_tag}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{region_tag}.dp_distortion_{CASE_POP}.png")
    plt.close()

    # LR histogram for eps=1.0 seed=0
    if region_tag in hist_cache:
        scores_member, scores_nonmember, thresh = hist_cache[region_tag]
        plt.figure(figsize=(6, 4))
        plt.hist(scores_member, bins=25, alpha=0.6, label=f"{CASE_POP} train (members)")
        plt.hist(scores_nonmember, bins=25, alpha=0.6, label=f"{CASE_POP} test (nonmembers)")
        plt.axvline(thresh, color="k", linestyle="--", linewidth=1, label="95% nonmember threshold")
        plt.xlabel("LR score")
        plt.ylabel("count")
        plt.title(f"LR score distributions — {region_tag} (eps=1.0)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(FIG_DIR / f"{region_tag}.lr_hist_eps_1p0_{CASE_POP}.png")
        plt.close()

print("Plots saved under", FIG_DIR.relative_to(PROJECT_ROOT))


Saved summary table -> results/tables/method3/method3_summary_MKK.csv
Plots saved under results/figures/method3
